In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Flatten, Conv1D
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
import pandas as pd
from CLayer.CLayer import *

In [3]:
import os
import numpy as np
from PIL import Image as im
import csv


def dense_to_one_hot(labels_dense):
    """Convert class labels from scalars to one-hot vectors."""
    num_classes = int(np.amax(labels_dense) + 1)
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot


class DataSet(object):
    def __init__(self, images, labels, one_hot=False):
        #images_2 is series
        assert images.shape[0] == labels.shape[0], ('images_1.shape: %s labels_1.shape: %s' % (images.shape, labels.shape))
        self._num_examples = images.shape[0]
        # Convert shape from [num examples, rows, columns, depth]
        # to [num examples, rows*columns] (assuming depth == 1)
        # assert images.shape[3] == 1
        #images = images.reshape(images.shape[0], images.shape[1] * images.shape[2] * images.shape[3])
        # Convert from [0, 255] -> [-1.0, 1.0].
        images = images.astype(np.float32)
        images = np.multiply(images, 1.0 / 127.5) - 1.
        self._images = images
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0

    @property
    def images(self):
        return self._images


    @property
    def labels(self):
        return self._labels

    @property
    def num_examples(self):
        return self._num_examples

    @property
    def epochs_completed(self):
        return self._epochs_completed

    def next_batch(self, batch_size):
        """Return the next `batch_size` examples from this data set."""
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            print("epoch " + str(self._epochs_completed))
            # Shuffle the data
            perm = np.arange(self._num_examples)
            np.random.shuffle(perm)
            self._images = self._images[perm]
            self._labels = self._labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        end = self._index_in_epoch
        return self._images[start:end], self._labels[start:end]


def load_data_from_pickle(data_file, label_file, image_shape):
    import pickle
    print(data_file)
    output = open(data_file, 'rb')
    labels = pickle.load(output)
    images = pickle.load(output)
    output.close()
    images = np.reshape(images, (np.shape(labels)[0], image_shape[0], image_shape[1], image_shape[2]))
    return images, labels

def load_data(data_file, label_file, image_shape, onehot):
    print(data_file)
    images = np.genfromtxt(data_file, delimiter=' ')
    labels = np.genfromtxt(label_file, usecols=(1), delimiter=' ')
    if onehot:
       labels = dense_to_one_hot(labels.astype(int))

    return images, labels

def load_data_from_file(data_file, label_file, image_shape, onehot):
    print(data_file)
    labelsall = np.genfromtxt(label_file, delimiter=' ', dtype=None)
    labelsshape = np.shape(labelsall)
    
    images = np.zeros((labelsshape[0], image_shape[0], image_shape[1], image_shape[2]))
    labels = np.zeros((labelsshape[0]))
    count = 0
    for line in labelsall:
       labels[count] = line[1]
       imagefile = im.open(data_file + line[0].decode("utf-8"))
       imagefile = imagefile.convert('RGB')
       images[count] = np.array(imagefile)
       imagefile.close()
       if count % 1000 == 0:
           print(count)
       count += 1
    if onehot:
       labels = dense_to_one_hot(labels.astype(int))

    return images, labels

def read_data_sets(train_file, train_label, shape, test_file="", test_label="", test_ratio=0.1, validation_ratio=0.0, pickle=True, boring=False, onehot=False):
    class DataSets(object):
        pass

    data_sets = DataSets()

    if (pickle):
        train_images, train_labels = load_data_from_pickle(train_file, train_label, shape)
        if test_file:
            test_images, test_labels = load_data_from_pickle(test_file, test_label, shape)
        else:
            test_size = int(test_ratio * float(train_labels.shape[0]))
            test_images = train_images[:test_size]
            test_labels = train_labels[:test_size]
            train_images = train_images[test_size:]
            train_labels = train_labels[test_size:]
    elif(boring):
        train_images, train_labels = load_data_from_file(train_file, train_label, shape, onehot)
        if test_file:
            test_images, test_labels = load_data_from_file(test_file, test_label, shape, onehot)
        else:
            test_size = int(test_ratio * float(train_labels.shape[0]))
            test_images = train_images[:test_size]
            test_labels = train_labels[:test_size]
            train_images = train_images[test_size:]
            train_labels = train_labels[test_size:]
    else:
        train_images, train_labels = load_data(train_file, train_label, shape, onehot)
        if test_file:
            test_images, test_labels = load_data(test_file, test_label, shape, onehot)
        else:
            test_size = int(test_ratio * float(train_labels.shape[0]))
            test_images = train_images[:test_size]
            test_labels = train_labels[:test_size]
            train_images = train_images[test_size:]
            train_labels = train_labels[test_size:]

    validation_size = int(validation_ratio * float(train_labels.shape[0]))
    validation_images = train_images[:validation_size]
    validation_labels = train_labels[:validation_size]

    train_images = train_images[validation_size:]
    train_labels = train_labels[validation_size:]

    data_sets.train = DataSet(train_images, train_labels)
    data_sets.validation = DataSet(validation_images, validation_labels)
    data_sets.test = DataSet(test_images, test_labels)
    
    print("data loaded")
    return data_sets

In [4]:
TRAINING_FILE = "../data/Unipen/1a-re-data.txt"
TRAINING_LABEL = "../data/Unipen/1a-re-labels.txt"
IMAGE_SHAPE = (50, 2)
data_sets = read_data_sets(TRAINING_FILE, TRAINING_LABEL, IMAGE_SHAPE, validation_ratio=0.0, pickle=False, boring=False, onehot=True)
train_data = (data_sets.train.images.reshape((-1, 50, 2)) + 1. ) * (127.5 / 127.)   # this input_data assumes images
train_labels = data_sets.train.labels
train_data.shape

../data/Unipen/1a-re-data.txt
data loaded


(11700, 50, 2)

In [5]:
print(train_data.shape)
print(train_labels.shape)

(11700, 50, 2)
(11700, 10)


In [37]:


tf.random.set_seed(1234)
model = Sequential([
    Input(train_data.shape[1:]),
    Conv1D(10,3, activation='relu'),
    Flatten(),
    Dense(10, activation='softmax')   
])

model.summary()
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.fit(train_data, train_labels, epochs=10, validation_split=0.3)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 48, 10)            70        
_________________________________________________________________
flatten_2 (Flatten)          (None, 480)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                4810      
Total params: 4,880
Trainable params: 4,880
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
256/256 [==============================] - 1s 3ms/step - loss: 1.4441 - accuracy: 0.6002 - val_loss: 0.9963 - val_accuracy: 0.7314
Epoch 2/20
256/256 [==============================] - 1s 3ms/step - loss: 0.8695 - accuracy: 0.7565 - val_loss: 0.7724 - val_accuracy: 0.7747
Epoch 3/20
256/256 [==============================] - 1s 2ms/step - loss: 0.7198 - accuracy: 0.793

In [ ]:


tf.random.set_seed(1234)
model = Sequential([
    Input(train_data.shape[1:]),
    DWA_CNN(10,3),
    Flatten(),
    Dense(10, activation='softmax')   
])

model.summary()
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.fit(train_data, train_labels, epochs=10, validation_split=0.3)


Tensor("map/while/stack_130:0", shape=(10,), dtype=float32)
Tensor("map/TensorArrayV2Stack/TensorListStack:0", shape=(48, 10), dtype=float32)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dwa_cnn (DWA_CNN)            (None, 48, 10)            70        
_________________________________________________________________
flatten (Flatten)            (None, 480)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                4810      
Total params: 4,880
Trainable params: 4,880
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
256/256 [==============================] - 2328s 9s/step - loss: 1.3262 - accuracy: 0.6604 - val_loss: 0.8856 - val_accuracy: 0.7434
Epoch 2/10
256/256 [==============================] - 2569s 10s/step - loss: 0.8043 - accuracy: 0.